In [1]:
import logging
import sys
import time
import random

In [2]:
#import settings
#import utils
import pandas as pd
import requests
import sys
import datetime
import pprint
from bs4 import BeautifulSoup

In [3]:
#Write a function to find the next Saturday
def getSaturday():
    d=datetime.date.today()
    d.day
    if d.weekday() == 5:
        d += datetime.timedelta(7)
        print('next saturday is',d.day)
    else:
        while d.weekday() != 5:
            d += datetime.timedelta(1)
    return d

In [4]:
#need to exe a week before the tee date
def weekbefore(sat_string):
    d = datetime.datetime.strptime(sat_string, '%m-%d-%Y')
    run_date = d-datetime.timedelta(7)
    return run_date

In [5]:
#find the index of the teetime in the json response. 
def findTeeIndex(tee_times,my_tee_time):
    ind_tee = 0
    for x in tee_times:
        if x.get('time')== my_tee_time:
            return ind_tee
        else:
            ind_tee = ind_tee+1

In [7]:
#day_saturday = getSaturday()
#sat_string = day_saturday.strftime("%m-%d-%Y")
#sat_string = '06-10-2021'

#For testing purposes input date
sat_string = input('What date would you like to book a time for: (MM-DD-2021)')
print(sat_string)

What date would you like to book a time for: (MM-DD-2021)06-22-2021
06-22-2021


In [8]:
run_date = weekbefore(sat_string)
run_date = run_date.strftime("%m-%d-%Y")
print(run_date)
#Tee times come out at 5:30 
time_run = '05:30'

06-15-2021


In [9]:
data = pd.read_csv('info.txt')

In [10]:
V_USERNAME = data['Login Info'][0]
V_PASSWORD = data['Login Info'][1]
V_WEBSITE = data['Login Info'][2]
V_COURSE_NAME = data['Login Info'][3]
V_COURSE_ID =data['Login Info'][4]
OPEN_TEE_TIMES_URL = V_WEBSITE+"/index.php/api/booking/times"
BOOK_URL = V_WEBSITE+"/index.php/api/booking/users/reservations"
LOGIN_URL = V_WEBSITE+"/index.php/api/booking/users/login"
DELETE_URL = V_WEBSITE+"/index.php/api/booking/users/reservations/"
PENDING_RESERVATION = V_WEBSITE+"/index.php/api/booking/pending_reservation"
cookies = data['Login Info'][5]

In [11]:
v_players = input('How many players')
v_players

How many players2


'2'

In [12]:
#Parameters
v_time = "morning"
v_holes = "18"
#v_players = "4"
v_booking_class = "false"
v_schedule_id = "6654"
v_specials_only = "0"
v_api_key = "no_limits"
v_date = sat_string
v_course_id = "21017"
v_schedule_ids={0:'0',1:'6463',2:'6654'}

In [13]:
LOGIN_PARAMS = {'api_key':v_api_key,'booking_class':v_booking_class,
               'course_id':v_course_id,'username':V_USERNAME,'password':V_PASSWORD}


In [14]:
response = requests.post(LOGIN_URL,data=LOGIN_PARAMS,cookies=cookies)
is_okay = response.status_code == requests.codes.ok
if is_okay:
    print("Logged in!")
else:
    response.raise_for_status()

Logged in!


In [15]:
#lets let things settle so its not so obvious we are a program 
#lets do different times so that we are not same exact time on the dot each week
x_rand = (random.random())
y_rand = (random.random())
time.sleep(x_rand)
time.sleep(y_rand)

In [16]:
payload = {"time":v_time, "date":v_date, "holes":v_holes, "players": v_players,
               "booking_class": v_booking_class, "schedule_id":v_schedule_id,
               "specials_only": v_specials_only, "api_key": v_api_key,"schedule_ids":v_schedule_ids}



In [17]:
resp = requests.get(OPEN_TEE_TIMES_URL, params=payload,cookies=cookies)
is_okay = resp.status_code == requests.codes.ok
if is_okay:
    print("Grabbed Tee Times!")
else:
    resp.raise_for_status()
open_tee_times = resp.json()

Grabbed Tee Times!


In [18]:
#resp.content

In [19]:
date_readable = v_date
print("There are "+str(len(open_tee_times))+" morning tee times available for "+date_readable+".")

There are 5 morning tee times available for 06-22-2021.


In [20]:
tee_times = open_tee_times

In [1]:
#tee_times

In [22]:
#FIND MY IDEAL TEE TIMES 8 or 9 AM 
times = []
times_8 = []
times_9 = []
my_time_available = False
best_time = None
for teetime in tee_times:
    times.append(teetime.get('time'))
    if "08:" in teetime.get('time'):
        times_8.append(teetime.get('time'))
    if "09:" in teetime.get('time'):
        my_time_available = True
        best_time = teetime
        times_9.append(teetime.get('time'))

In [23]:
#tee_times
times_8

['2021-06-22 08:00', '2021-06-22 08:10']

In [24]:
type(tee_times)

list

In [25]:
#if 9 or later is available book that one. If not look at 8AM
if my_time_available == True: 
    my_tee_time = best_time
else: 
    my_tee_time = times_8[0]
    if times_8 == []:
        no_tee_time = True
    else:
        pass


In [26]:
#my_tee_time ='2021-06-05 08:27' 
my_tee_time

'2021-06-22 08:00'

In [2]:
ind_tee = findTeeIndex(tee_times,my_tee_time)
#ind_tee

In [28]:
x_rand = (random.random())
y_rand = (random.random())
time.sleep(x_rand)
time.sleep(y_rand)

In [29]:
#pending reservation parameters
p_Time = my_tee_time[:10]+' '+my_tee_time[11:]
p_holes = tee_times[ind_tee].get('holes')
p_players = v_players
p_carts = "false"
p_schedule_id = str(tee_times[ind_tee].get('schedule_id'))
p_teesheet_side_id = str(tee_times[ind_tee].get('teesheet_side_id'))
p_course_id = str(tee_times[ind_tee].get('course_id'))
p_booking_class_id = str(tee_times[ind_tee].get('booking_class_id'))
p_duration = '1'

In [30]:
payload_p = {"time":p_Time,'holes':p_holes,'players':p_players,'carts':p_carts,'schedule_id':p_schedule_id,
             'teesheet_side_id':p_teesheet_side_id,'course_id':p_course_id,'booking_class_id':p_booking_class_id,'duration':p_duration}

In [3]:
# To book TEE time need a temp reservation ID, can use json response for input to final request to book. 
resp_p = requests.get(PENDING_RESERVATION, params=payload_p,cookies=cookies)
is_okay = resp_p.status_code == requests.codes.ok
if is_okay:
    print("Temp Reservation")
else:
    resp_p.raise_for_status()
r_reservation_id = resp_p.get("reservation_id")

In [56]:
x_rand = (random.random())
y_rand = (random.random())
time.sleep(x_rand)
time.sleep(y_rand)

In [4]:
green_fee = dict1.get('green_fee')
total_cost = green_fee * int(p_players)

In [54]:
dict1 = tee_times[ind_tee]
dict2 = {'players':p_players, 'carts':False, 'promo_code':'', 'promo_discount':0, 'player_list':False, 'duration':1, 'notes':[], 'customer_message':'', 'guest_cart_fee':False, 'total':total_cost, 'purchased':False, 'pay_players':p_players, 'pay_carts':False, 'pay_total':total_cost, 'pay_subtotal':total_cost, 'paid_player_count':0, 'discount_percent':0, 'discount':0, 'details':'', 'pending_reservation_id':r_reservation_id, 'allow_mobile_checkin':0, 'foreup_trade_discount_information':[], 'airQuotesCart':[{"type":"item","description":"Green Fee","price":green_fee,"quantity":p_players,"subtotal":total_cost}], 'preTaxSubtotal':total_cost, 'estimatedTax':0, 'subtotal':total_cost, 'available_duration':'null', 'increment_amount':'null', 'credit_card_id':str(data['Login Info'][6])}
dict1.update(dict2) 

In [ ]:
resp_reserve = requests.get(BOOK_URL, params=dict1,cookies=cookies)
is_okay = resp_reserve.status_code == requests.codes.ok
if is_okay:
    print("BOOKED TIME")
else:
    resp_reserve.raise_for_status()


In [5]:
del_url = BOOK_URL +'/'+ r_reservation_id
resp_del = requests.delete(del_url)
is_okay = resp_del.status_code == requests.codes.ok
if is_okay:
    print("Temp Reservation")
else:
    resp_del.raise_for_status()